## Agile Hardware Design
***
# Introduction to Functional Programming Primer

<img src="./images/chisel_logo.svg" alt="agile hardware design logo" style="width:20%;float:right"/>

By Peter Hanping Chen based on
1. UCB Bootcamp load-ivy.sc
   - https://github.com/freechipsproject/chisel-bootcamp
2. UCSC Prof. Scott Beamer, sbeamer@ucsc.edu
   - removed chisel_deps.sc due to dependencies error
   - [CSE 228A](https://classes.soe.ucsc.edu/cse228a/Winter24/)
              

## Plan for Today

* _Big Idea:_ applying functions to collections of elements
* Anonymous functions in Scala
* Scala `map`, `foreach`, `zip` operators
* Chisel example

## Loading The Chisel Library Into a Notebook

In [42]:
//interp.load.module(os.Path(s"${System.getProperty("user.dir")}/../resource/chisel_deps.sc"))
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
//val path = System.getProperty("user.dir") + "/source/chisel_deps.sc"
println("path: "+path)
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/010_IntroFP/001_Code/source/load-ivy.sc
Compiling /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/010_IntroFP/001_Code/Main.sc

Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chisel-iotesters_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/chiseltest_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/dsptools_2.12/maven-metadata.xml
Checking https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml
Checked https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl-diagrammer_2.12/maven-metadata.xml


Compiling /home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/010_IntroFP/001_Code/Main.sc #2

path: String = "/home/peter/AIU/AIU_CS800_Chisel/500_UCSC_HWD/010_IntroFP/001_Code/source/load-ivy.sc"

In [43]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

import chisel3._

import chisel3.util._

import chiseltest._

import chiseltest.RawTester.test

## Why Use Functional Programming with Chisel?

* Chisel's power comes from its ability to make parameterized hardare generators

* Functional programming (FP) operations ease dealing with _functions over collections_
  * Using standard features/patterns improves productivity, readability, and correctness

* Being forced to break problem into standard patterns may help with reasoning
  * Additionally, compiler may be able to spot more errors

* Be mindful of side-effects and how to clearly convey them in code
  * FP operations typically intended for side-effect free programming
  * Chisel operations often have deliberate side effects (e.g. connecting things)

## Motivation for Working Over Collections

* Arguably, much of programming (and hardware design) works over collections rather than scalar values
  * Collections aggregate similar things

* Often, we want to apply an operation to everything in the collection
  * Traditionally, we use iteration (e.g. `for` loops)

* _Problem:_ every usage has to reinvent wheel

* What if ...
  * We could recognize _patterns_ of function application and use those?
  * We could use the compiler to check compliance with those patterns?

## Solution - Reuse through Patterns

* Note these operations are not core language constructs, but instead methods defined for these collections
* Only some operations covered today (more to come)

<img src="./images/map+foreach.svg" alt="map & foreach viz" style="width:80%;align:left"/>

## Scala Anonymous Functions
* Technically called _function literals_
* Can bind to a name, but often will use within other construct and never name it explicitly (_anonymous_)
* Syntax - argument list in parentheses on left, `=>`, function body on right
```scala
    (x: Int) => x + 1
```

In [44]:
// Define an anonymous function (x: Int) without function name and with alias "inc"
val inc = (x: Int) => x + 1
println("*** Pass parameter 2 to anonymous function with alias \"inc\" ***")
println("  inc(2): " + inc(2))

// Define a normal function name "inc2"
def inc2(x: Int) = x+1
println("*** Pass parameter 2 to a normal function with name \"inc2\" ***")
println("  inc2(2): " + inc(2))

// Define an anynomous function (a: Int, b: Int) with name sum2
val sum2 = (a: Int, b: Int) => a+b
println ("*** Pass parameters: (2, 3) to anonymous function with alias \"sum2\" ***")
println("  sum2(2, 3): " + sum2(2, 3))

*** Pass parameter 2 to anonymous function with alias "inc" ***
  inc(2): 3
*** Pass parameter 2 to a normal function with name "inc2" ***
  inc2(2): 3
*** Pass parameters: (2, 3) to anonymous function with alias "sum2" ***
  sum2(2, 3): 5


inc: Int => Int = ammonite.$sess.cmd43$Helper$$Lambda$4495/900098489@4d54591
defined function inc2
sum2: (Int, Int) => Int = ammonite.$sess.cmd43$Helper$$Lambda$4496/1051160266@8965ecb

## `map` in Scala

```scala
l map f
```

* Applies given function to each element and returns result as new collection
* Should not make assumptions about order in which it is applied

In [45]:
def inc(x: Int) = x+1

val l = 0 until 5
println("**** l.map(inc): ****")
println(l.map(inc))
println ("**** l map inc: ****")
println(l map inc)
println ("**** l map { i => inc(i) }: ****")
println(l map { i => inc(i) })
println ("**** l map { i => i + 1 }: ****")
println (l map { i => i + 1 })

**** l.map(inc): ****
Vector(1, 2, 3, 4, 5)
**** l map inc: ****
Vector(1, 2, 3, 4, 5)
**** l map { i => inc(i) }: ****
Vector(1, 2, 3, 4, 5)
**** l map { i => i + 1 }: ****
Vector(1, 2, 3, 4, 5)


defined function inc
l: Range = Range(0, 1, 2, 3, 4)

## `foreach` in Scala

```scala
l foreach f
```

* Applies given function to each element (like map), but does not return anything (unlike map)
* Useful for indicating intent is the side effect and not the result

In [46]:
val l = 0 until 5
println ("**** l foreach println: ****")
l foreach println

**** l foreach println: ****
0
1
2
3
4


l: Range = Range(0, 1, 2, 3, 4)

## Using `map` and `foreach` in Chisel

* Operations available on both Scala collections (e.g. `Seq`) and Chisel aggregates (e.g. `Vec`)

In [57]:
class ConstOut(numElems: Int, const: Int) extends Module {
    val io = IO(new Bundle {
        val out = Output(Vec(numElems, UInt()))
    })

    val seqOfInts = 0 until numElems
    val seqOfUInts = seqOfInts map { i => i.U }
    //println("**** seqOfUInts: ****")
    //println(seqOfUInts)
    io.out foreach { o => o := const.U }
    //println("**** io.out: ****")
    //println(io.out)
}

defined class ConstOut

In [58]:
//printVerilog(new ConstOut(2,8))
println(getVerilog(new ConstOut(2,8)))

Elaborating design...
Done elaborating.
module ConstOut(
  input        clock,
  input        reset,
  output [3:0] io_out_0,
  output [3:0] io_out_1
);
  assign io_out_0 = 4'h8; // @[cmd56.sc 10:29]
  assign io_out_1 = 4'h8; // @[cmd56.sc 10:29]
endmodule



## Scala Tuples

* Can group together heterogeneous things
  * Doesn't name members, but can index them numerically (**starts from 1**)
  * Often can pattern match (with `case`) or assign to access members

* Best when number of things is small and producer/consumers are nearby
  * Suggest case class to explicitly name members (for readability)
  * Suggest collection for many elements (for manageability)

In [66]:
val t1 = (2,3)
val t2 = ("My", 8)
println ("t1._1: " + t1._1)
println ("t1._2: " + t1._2)
println()
println ("t2._1: " + t2._1)
println ("t2._2: " + t2._2)
println ()
val (a,b) = t1
println ("a: " + a)
println ("b: " + b)

t1._1: 2
t1._2: 3

t2._1: My
t2._2: 8

a: 2
b: 3


t1: (Int, Int) = (2, 3)
t2: (String, Int) = ("My", 8)
a: Int = 2
b: Int = 3

## `zip` in Scala

```scala
l1 zip l2
```

* Pairs up elements with elements of another collection
* Commonly used to join together collections before applying other operations
* _Note:_ If collections have different sizes, result is the minimum size

In [73]:
val l1 = 0 until 5
println ("****l1 zip l1:****")
println(l1 zip l1)
println ("****l1 zip Seq(8):****")
println(l1 zip Seq(8))
println ("**** l1 ****")
println (l1)
println ("****l1 zip l1 map {case (a,b) => a+b}:****")
println(l1 zip l1 map {case (a,b) => print (" a: " + a); println(" b: " + b); a+b})

****l1 zip l1:****
Vector((0,0), (1,1), (2,2), (3,3), (4,4))
****l1 zip Seq(8):****
Vector((0,8))
**** l1 ****
Range 0 until 5
****l1 zip l1 map {case (a,b) => a+b}:****
 a: 0 b: 0
 a: 1 b: 1
 a: 2 b: 2
 a: 3 b: 3
 a: 4 b: 4
Vector(0, 2, 4, 6, 8)


l1: Range = Range(0, 1, 2, 3, 4)

## `zip` Diagram

<img src="images/zip.svg" alt="zip viz" style="width:70%;align: left"/>

## Chisel Example Using `foreach` and `zip`

In [74]:
class VecAbs(numElems: Int, width: Int) extends Module {
    val io = IO(new Bundle {
        val in = Input(Vec(numElems, SInt(width.W)))
        val out = Output(Vec(numElems, SInt(width.W)))
    })

    def abs(x: SInt): SInt = Mux(x < 0.S, -x, x)

//     for (i <- 0 until numElems) {
//         io.out(i) := abs(io.in(i))
//     }

    io.out.zip(io.in) foreach { case (o,i) => o := abs(i) }
}

defined class VecAbs

In [52]:
//printVerilog(new VecAbs(2,8))
println(getVerilog((new VecAbs(2,8))))

Elaborating design...
Done elaborating.
module VecAbs(
  input        clock,
  input        reset,
  input  [7:0] io_in_0,
  input  [7:0] io_in_1,
  output [7:0] io_out_0,
  output [7:0] io_out_1
);
  wire [7:0] _T_3 = 8'sh0 - $signed(io_in_0); // @[cmd50.sc 7:43]
  wire [7:0] _T_8 = 8'sh0 - $signed(io_in_1); // @[cmd50.sc 7:43]
  assign io_out_0 = $signed(io_in_0) < 8'sh0 ? $signed(_T_3) : $signed(io_in_0); // @[cmd50.sc 7:33]
  assign io_out_1 = $signed(io_in_1) < 8'sh0 ? $signed(_T_8) : $signed(io_in_1); // @[cmd50.sc 7:33]
endmodule



## Scala (fuction) Placeholders

* Able to make function literals even more concise by not explicitly naming arguments and then using them
* Use `_` in place of argument, and each use advances to next argument
* Use **CAREFULLY** to shorten code to improve readability
  * If intent not immediately clear, fall back to explicitly naming arguments

In [53]:
val l = 0 until 5
l map { i => i + 1 }
l map { _ + 1 }

l: Range = Range(0, 1, 2, 3, 4)
res52_1: collection.immutable.IndexedSeq[Int] = Vector(1, 2, 3, 4, 5)
res52_2: collection.immutable.IndexedSeq[Int] = Vector(1, 2, 3, 4, 5)

## Redoing Our Arbiter with FP (1/2)

In [54]:
class MyArb(numPorts: Int, n: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(n.W))))
        val out = Decoupled(UInt(n.W))
    })
    require (numPorts > 0)
    val inValids = Wire(Vec(numPorts, Bool()))
    val inBits   = Wire(Vec(numPorts, UInt(n.W)))
    val chosenOH = PriorityEncoderOH(inValids)
    for (p <- 0 until numPorts) {
        io.req(p).ready := chosenOH(p) && io.out.fire
        inValids(p) := io.req(p).valid
        inBits(p) := io.req(p).bits
    }
    io.out.valid := inValids.asUInt.orR
    io.out.bits := Mux1H(chosenOH, inBits)
}

defined class MyArb

## Redoing Our Arbiter (2/2)

In [55]:
class MyArb(numPorts: Int, n: Int) extends Module {
    val io = IO(new Bundle {
        val req = Flipped(Vec(numPorts, Decoupled(UInt(n.W))))
        val out = Decoupled(UInt(n.W))
    })
    require (numPorts > 0)
    val inValids = io.req map { _.valid }
    io.out.valid := VecInit(inValids).asUInt.orR
    val chosenOH = PriorityEncoderOH(inValids)
    io.out.bits := Mux1H(chosenOH, io.req map { _.bits })
    io.req.zip(chosenOH) foreach { case (i, c) => i.ready := c && io.out.fire}
}

defined class MyArb